# Assembling

This notebook is used to assemble a processable h5ad object for subsequent notebooks.

## Fill in input data, output and settings

In [1]:
####################### TEST NAME ###################################
test = 'Esophagus'
################### DEFINING STRATEGY ###############################
from_h5ad = True #option 1
assembling_10_velocity = False #option 2
assembling_10_public = False #option 3
convert_seurat = False #option 4

###################### INPUT DATA ###################################

#For option 1: The path to an existing .h5ad file
h5ad_files = ['/home/rstudio/data/anndata/Esophagus.h5ad']
#Manually set existing QC Columns
n_features_by_counts = None
log1p_n_features_by_counts = None
total_counts = None
log1p_total_counts = None
mean_insertsize = None
insertsize_count = None
n_total_fragments = None
n_fragments_in_promoters = None
pct_fragments_in_promoters = None
blacklist_overlaps = None
#total_number_of_fragments
TN = 'TN'
#uniquely_mapped_fragments
UM = 'UM'
#properly_paired_fragments
PP = 'PP'
#uniq_fragments
UQ = 'UQ'
#chrM_fragments
CM = 'CM'

#################### Calc QC Columns ###############################
#set available number of threads
n_threads = 8

#if the mean_insertsize should be calculated specify either a BAM file or a fragments file:
fragments_file = '/home/rstudio/data/bamfiles/fragments_Esophagus.bed'
bam_file = '/home/rstudio/data/bamfiles/sorted_Esophagus.bam'

#specify promotors_gtf if another promoter than the fixed ones should be used
#if promotors is None please specify species
#promoters_gtf =  '/mnt/flatfiles/organisms/new_organism/homo_sapiens/104/homo_sapiens.104.promoters2000.gtf'
promoters_gtf = '/home/rstudio/data/homo_sapiens.104.promoters2000.gtf'
species = None

##################### OUTPUT DATA ###################################
output_dir = '/home/rstudio/processed_data'

## Import modules

In [2]:
# sctoolbox modules 
import sctoolbox.atac_tree as sub_tree
import sctoolbox.creators as cr
import sctoolbox.fragment_length as fragments
import sctoolbox.atac as atac
import sctoolbox.calc_overlap_pct as overlap
import sctoolbox.analyser 
from sctoolbox.qc_filter import *
from sctoolbox.atac_utils import *
# import episcanpy
import episcanpy as epi

## Setup path handling object 

In [3]:
# make an instance of the class
tree = sub_tree.ATAC_tree()
# set processing/output directory
tree.processing_dir = output_dir
# set sample/experiment.. 
tree.run = test

all directories existing
all directories existing


## Read in data

### Option 1: Read from .h5ad

In [4]:
qc_columns = {}
qc_columns["n_features_by_counts"] = n_features_by_counts
qc_columns["log1p_n_features_by_count"] = log1p_n_features_by_counts
qc_columns["total_counts"] = total_counts
qc_columns["log1p_total_counts"] = log1p_total_counts
qc_columns["mean_insertsize"] = mean_insertsize
qc_columns['n_total_fragments'] = n_total_fragments
qc_columns['n_fragments_in_promoters'] = n_fragments_in_promoters
qc_columns['pct_fragments_in_promoters'] = pct_fragments_in_promoters
qc_columns["blacklist_overlaps"] = blacklist_overlaps
qc_columns["TN"] = TN
qc_columns["UM"] = UM
qc_columns["PP"] = PP
qc_columns["UQ"] = UQ
qc_columns["CM"] = CM

In [5]:
if from_h5ad:
    
    adata = assemble_from_h5ad(h5ad_files=h5ad_files, qc_columns=qc_columns)

add existing adata.obs columns to infoprocess:

TN:TN
UM:UM
PP:PP
UQ:UQ
CM:CM
setting adata.obs.index = adata.obs[barcode]


In [ ]:
if from_h5ad:
    
    sample = 'sample1'
    
    adata = epi.read_h5ad(h5ad_path)
    
    #Add information to the infoprocess
    cr.build_infor(adata, "Input_for_assembling", h5ad_path)
    cr.build_infor(adata, "Strategy", "Read from h5ad")
    
    print('add existing adata.obs columns to infoprocess:')
    print()
    for key, value in qc_columns.items():
        if value is not None:
            print(key + ':' + value)
            if value in adata.obs.columns:
                build_legend(adata, key, value)
            else:
                print('column:  ' + value + ' is not in adata.obs')
    
    if not adata.obs.index.name == "barcode":
        print('setting adata.obs.index = adata.obs[barcode]')
        adata.obs = adata.obs.set_index("barcode")
    else:
        print('barcodes are already the index')
    
    adata.obs = adata.obs.assign(sample=sample)
    adata.obs = adata.obs.assign(file=h5ad_path)
    

## Inspect adata

In [6]:
display(adata)

AnnData object with n_obs × n_vars = 73652 × 150152
    obs: 'TN', 'UM', 'PP', 'UQ', 'CM', 'file', 'sample'
    uns: 'infoprocess', 'color_set', 'legend'

In [7]:
adata.obs

,TN,UM,PP,UQ,CM,file,sample
barcode,,,,,,,
AAACTACCAGAAACCCGAGATA_sample1,33.0,18.0,18.0,15.0,0.0,/home/rstudio/data/anndata/Esophagus.h5ad,sample1
AAACTACCAGAAACCTAAGTGG_sample1,52.0,36.0,35.0,32.0,1.0,/home/rstudio/data/anndata/Esophagus.h5ad,sample1
AAACTACCAGAAACGGATCAGT_sample1,27.0,19.0,19.0,19.0,0.0,/home/rstudio/data/anndata/Esophagus.h5ad,sample1
AAACTACCAGAAACGTCCCGTT_sample1,6223.0,5231.0,5213.0,3779.0,18.0,/home/rstudio/data/anndata/Esophagus.h5ad,sample1
AAACTACCAGAAACTAGCCCTA_sample1,41.0,29.0,29.0,26.0,0.0,/home/rstudio/data/anndata/Esophagus.h5ad,sample1
...,...,...,...,...,...,...,...
TTCCATCCTCTTTCGCGTGTAA_sample1,9.0,6.0,6.0,6.0,0.0,/home/rstudio/data/anndata/Esophagus.h5ad,sample1
TTCCATCCTCTTTCTGCAGACT_sample1,10.0,6.0,6.0,6.0,1.0,/home/rstudio/data/anndata/Esophagus.h5ad,sample1
TTCCATCCTCTTTCTGGCGCAG_sample1,17.0,5.0,5.0,5.0,1.0,/home/rstudio/data/anndata/Esophagus.h5ad,sample1


In [8]:
display(adata.var)

""
0
1
2
3
4
...
150147
150148
150149
150150


In [9]:
display(adata.obs)

,TN,UM,PP,UQ,CM,file,sample
barcode,,,,,,,
AAACTACCAGAAACCCGAGATA_sample1,33.0,18.0,18.0,15.0,0.0,/home/rstudio/data/anndata/Esophagus.h5ad,sample1
AAACTACCAGAAACCTAAGTGG_sample1,52.0,36.0,35.0,32.0,1.0,/home/rstudio/data/anndata/Esophagus.h5ad,sample1
AAACTACCAGAAACGGATCAGT_sample1,27.0,19.0,19.0,19.0,0.0,/home/rstudio/data/anndata/Esophagus.h5ad,sample1
AAACTACCAGAAACGTCCCGTT_sample1,6223.0,5231.0,5213.0,3779.0,18.0,/home/rstudio/data/anndata/Esophagus.h5ad,sample1
AAACTACCAGAAACTAGCCCTA_sample1,41.0,29.0,29.0,26.0,0.0,/home/rstudio/data/anndata/Esophagus.h5ad,sample1
...,...,...,...,...,...,...,...
TTCCATCCTCTTTCGCGTGTAA_sample1,9.0,6.0,6.0,6.0,0.0,/home/rstudio/data/anndata/Esophagus.h5ad,sample1
TTCCATCCTCTTTCTGCAGACT_sample1,10.0,6.0,6.0,6.0,1.0,/home/rstudio/data/anndata/Esophagus.h5ad,sample1
TTCCATCCTCTTTCTGGCGCAG_sample1,17.0,5.0,5.0,5.0,1.0,/home/rstudio/data/anndata/Esophagus.h5ad,sample1


## Calculate missing columns

### 1. calculate scanpy qc_metrics (n_features)

In [ ]:
if qc_columns['n_features_by_counts'] is None or qc_columns['log1p_n_features_by_counts'] is None:
    adata = analyser.calculate_qc_metrics(adata, var_type='features')
    
    qc_columns['n_features_by_count'] = 'n_features_by_count'
    qc_columns['log1p_n_features_by_counts'] = 'log1p_n_features_by_counts'
    qc_columns['total_counts'] = 'total_counts'
    qc_columns['log1p_total_counts'] = 'log1p_total_counts'
    
    build_legend(adata, 'n_features_by_count', 'n_features_by_count')
    build_legend(adata, 'log1p_n_features_by_counts', 'log1p_n_features_by_counts')
    build_legend(adata, 'total_counts', 'total_counts')
    build_legend(adata, 'log1p_total_counts', 'log1p_total_counts')

adata.obs

### 2. calc mean insertsize and count if missing 

In [ ]:
if qc_columns['mean_insertsize'] is None or qc_columns['insertsize_count'] is None:
    atac.add_insertsize(adata, fragments=fragments_file) # or use bam=bam_file instead of fragments
    
    # update column dictionary
    qc_columns['mean_insertsize'] = 'mean_insertsize'
    qc_columns['insertsize_count'] = 'insertsize_count'
    
    # update infoprocess table
    build_legend(adata, 'mean_insertsize', 'mean_insertsize')
    build_legend(adata, 'insertsize_count', 'insertsize_count')
    
adata.obs

In [ ]:
atac.plot_insertsize(adata, barcodes=None)

### 3. promotor enrichment 

In [ ]:
if qc_columns['n_fragments_in_promoters'] is None:
    overlap.pct_fragments_in_promoters(adata, gtf_file=promoters_gtf, species=species, bam_file=bam_file, cb_col=None, nproc=1)
    
    # n_total_fragments, n_fragments_in_promoters, pct_fragments_in_promoters
    # update column dictionary
    qc_columns['n_total_fragments'] = 'n_total_fragments'
    qc_columns['n_fragments_in_promoters'] = 'n_fragments_in_promoters'
    qc_columns['pct_fragments_in_promoters'] = 'pct_fragments_in_promoters'
    # update infoprocess table
    build_legend(adata, 'n_total_fragments', 'n_total_fragments')
    build_legend(adata, 'n_fragments_in_promoters', 'n_fragments_in_promoters')
    build_legend(adata, 'pct_fragments_in_promoters', 'pct_fragments_in_promoters')
    

## Inspect adata.obs

In [ ]:
adata.obs

## Save adata to .h5ad

In [ ]:
adata_output = tree.assembled_anndata
adata_output

In [ ]:
#Saving the data
cr.build_infor(adata, "Test_number", test)
cr.build_infor(adata, "Anndata_path", output_dir)

adata_output = tree.assembled_anndata
adata.write(filename=adata_output)